<a href="https://colab.research.google.com/github/JacobAdamson5/hotels/blob/main/Copy_of_hotels2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance --upgrade --no-cache-dir
!pip install openpyxl
!pip install schedule
!pip install plotly
!pip install pytz
import yfinance as yf
import pandas as pd
import time # Import the time module
import matplotlib.pyplot as plt
import numpy as np
!pip install seaborn
import seaborn as sns
import plotly.express as px
import schedule
import datetime
import pytz

# prompt: Show the graph with the existing data prior to the scheduled execution to retrieve new data
hotels = pd.read_csv('hotels.csv')
sp500 = pd.read_csv('sp500.csv')

correlation = hotels['Daily Return'].corr(sp500['Daily Return'])

hotelfig = px.line(title = 'Adamson Financial Hotels Index Returns')
hotelfig.add_scatter(x = hotels['Date'][:1256], y = hotels['AF Price'][:1256], name = 'AF Hotels Index', yaxis='y1')
hotelfig.add_scatter(x = sp500['Date'][:1256], y = sp500['Price'][:1256], name = 'S&P 500', yaxis='y2')

hotelfig.update_layout(
    title_x=0.5,
    yaxis=dict(title='AF Hotels Index', side='left', range=[50,210]),
    yaxis2=dict(title='S&P 500', overlaying='y', side='right', range=[2000,6200]),
    xaxis=dict(title='Date'),
    legend=dict(x=0.01, y=0.99)
)


hotelfig.show()

most_recent_hotels = hotels.iloc[0, :] # Get last row of data frame

# Create a table for the most recent hotel returns
from IPython.display import HTML, display

recent_hotels_table_data = [
        ['Daily Return', '5D Return', 'YTD Return', 'Correlation with S&P 500'],
          #Insert most recent data here
        [most_recent_hotels['Daily Return'], most_recent_hotels['5D Return'], most_recent_hotels['YTD Return'], correlation]
    ]

recent_hotels_table = pd.DataFrame(recent_hotels_table_data[1:], columns=recent_hotels_table_data[0])

# Convert relevant columns to percentages and round to two decimal places
for col in ['Daily Return', '5D Return', 'YTD Return']:
    recent_hotels_table[col] = (recent_hotels_table[col] * 100).round(2).astype(str) + '%'

# Convert correlation to percentage, round it, and append % symbol
recent_hotels_table['Correlation with S&P 500'] = (recent_hotels_table['Correlation with S&P 500']).round(2)

# Use HTML to center the table
display(HTML(f"""
<div style="text-align:center;">
<table style="width:88%; border-collapse: collapse;">
  <thead>
    <tr>
      <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">Daily Return</th>
      <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">5D Return</th>
      <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">YTD Return</th>
      <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">Correlation with S&P 500</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['Daily Return'] * 100:.2f}%</td>
      <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['5D Return'] * 100:.2f}%</td>
      <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['YTD Return'] * 100:.2f}%</td>
      <td style="border: 1px solid black; padding: 4px;">{correlation:.2f}</td>
    </tr>
  </tbody>
</table>
</div>
"""))


def job():
  hotels = pd.read_csv('hotels.csv')
  sp500 = pd.read_csv('sp500.csv')

  from datetime import date, timedelta

  today = date.today()
  today_str = today.strftime('%Y-%m-%d')

  new_row = pd.DataFrame({'Date': [today_str]
                        })
  hotels = pd.concat([new_row, hotels], ignore_index=True)

  import numpy as np
  tickers = ['CHH', 'H', 'HLT', 'IHG', 'MAR', 'WH']

  for ticker in tickers:
    try:
      stock_info = yf.Ticker(ticker)
      market_cap = stock_info.info.get('marketCap')
      if market_cap:
        hotels.loc[0, ticker] = market_cap
      else:
        print(f"Market cap not found for {ticker}")
        hotels.loc[0, ticker] = np.nan # or some other default value
    except Exception as e:
      print(f"Error retrieving data for {ticker}: {e}")
      hotels.loc[0, ticker] = np.nan # or some other default value

  time.sleep(1) # Add a small delay to avoid overloading the API

   # Calculate the sum of specified columns for the new row (index 0)
  columns_to_sum = ['CHH', 'H', 'HLT', 'IHG', 'MAR', 'WH']
  hotels.loc[0, 'Total'] = hotels.loc[0, columns_to_sum].sum()

  hotels.loc[0, 'AF Price'] = hotels.loc[0, 'Total']/1000000000
  hotels.loc[0, 'Daily Return'] = hotels.loc[0, 'AF Price']/hotels.loc[1, 'AF Price']-1
  hotels.loc[0, '5D Return'] = hotels.loc[0, 'AF Price']/hotels.loc[5, 'AF Price']-1
  hotels.loc[0, 'YTD Return'] = hotels.loc[0, 'AF Price'] / hotels.loc[hotels['Date'] == '2024-12-31', 'AF Price'].values[0] - 1

  sp500 = pd.concat([new_row, sp500], ignore_index=True)

  # prompt: In sp500, update the Price column in row 0 with the previous close of the S&P 500

  try:
    stock_info = yf.Ticker('^GSPC')
    prev_close = stock_info.info.get('previousClose')
    if prev_close:
      sp500.loc[0, 'Price'] = prev_close
    else:
      print(f"Price not found for {'^GSPC'}")
      sp500.loc[0, '^GSPC'] = np.nan # or some other default value
  except Exception as e:
    print(f"Error retrieving data for {'^GSPC'}: {e}")
    sp500.loc[0, '^GSPC'] = np.nan # or some other default value

  sp500.loc[0, 'Daily Return'] = sp500.loc[0, 'Price']/sp500.loc[1, 'Price']-1
  sp500.loc[0, '5D Return'] = sp500.loc[0, 'Price']/sp500.loc[5, 'Price']-1
  sp500.loc[0, 'YTD Return'] = sp500.loc[0, 'Price'] / sp500.loc[sp500['Date'] == '2024-12-31', 'Price'].values[0] - 1

  # Calculate the correlation between the daily returns
  correlation = hotels['Daily Return'].corr(sp500['Daily Return'])

  hotelfig = px.line(title = 'Adamson Financial Hotels Index Returns')
  hotelfig.add_scatter(x = hotels['Date'][:1256], y = hotels['AF Price'][:1256], name = 'AF Hotels Index', yaxis='y1')
  hotelfig.add_scatter(x = sp500['Date'][:1256], y = sp500['Price'][:1256], name = 'S&P 500', yaxis='y2')

  hotelfig.update_layout(
      title_x=0.5,
      yaxis=dict(title='AF Hotels Index', side='left', range=[50,210]),
      yaxis2=dict(title='S&P 500', overlaying='y', side='right', range=[2000,6200]),
      xaxis=dict(title='Date'),
      legend=dict(x=0.01, y=0.99)
)


  hotelfig.show()

# Assuming your 'hotels' DataFrame has columns 'Daily Return', '5D Return', and 'YTD Return'
# and a 'Hotel Name' column to identify the hotels.
# Get the most recent data for the hotels. If needed you can use .tail(1) as well
  most_recent_hotels = hotels.iloc[0, :] # Get last row of data frame

# Create a table for the most recent hotel returns
  from IPython.display import HTML, display

  recent_hotels_table_data = [
         ['Daily Return', '5D Return', 'YTD Return', 'Correlation with S&P 500'],
          #Insert most recent data here
          [most_recent_hotels['Daily Return'], most_recent_hotels['5D Return'], most_recent_hotels['YTD Return'], correlation]
    ]

  recent_hotels_table = pd.DataFrame(recent_hotels_table_data[1:], columns=recent_hotels_table_data[0])

# Convert relevant columns to percentages and round to two decimal places
  for col in ['Daily Return', '5D Return', 'YTD Return']:
     recent_hotels_table[col] = (recent_hotels_table[col] * 100).round(2).astype(str) + '%'

# Convert correlation to percentage, round it, and append % symbol
  recent_hotels_table['Correlation with S&P 500'] = (recent_hotels_table['Correlation with S&P 500']).round(2)

# Use HTML to center the table
  display(HTML(f"""
  <div style="text-align:center;">
  <table style="width:88%; border-collapse: collapse;">
    <thead>
      <tr>
        <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">Daily Return</th>
        <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">5D Return</th>
       <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">YTD Return</th>
       <th style="word-wrap: break-word; width: 22%; border: 1px solid black; padding: 4px;">Correlation with S&P 500</th>
     </tr>
   </thead>
   <tbody>
      <tr>
       <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['Daily Return'] * 100:.2f}%</td>
        <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['5D Return'] * 100:.2f}%</td>
       <td style="border: 1px solid black; padding: 4px;">{most_recent_hotels['YTD Return'] * 100:.2f}%</td>
        <td style="border: 1px solid black; padding: 4px;">{correlation:.2f}</td>
      </tr>
   </tbody>
  </table>
  </div>
  """))




# Schedule the job to run at 4:30 PM EST every weekday
schedule.every().monday.at("16:30").do(job)
schedule.every().tuesday.at("16:30").do(job)
schedule.every().wednesday.at("16:30").do(job)
schedule.every().thursday.at("16:30").do(job)
schedule.every().friday.at("16:30").do(job)


while True:
    schedule.run_pending()
    time.sleep(3600)  # Check every 1 hour


Daily Return,5D Return,YTD Return,Correlation with S&P 500
-1.23%,-10.86%,-11.58%,0.64


In [ ]:
# prompt: Show the graph with the existing data prior to the scheduled execution to retrieve new data

hotelfig = px.line(title = 'Adamson Financial Hotels Index Returns')
hotelfig.add_scatter(x = hotels['Date'], y = hotels['AF Price'], name = 'AF Hotels Index', yaxis='y1')
hotelfig.add_scatter(x = sp500['Date'], y = sp500['Price'], name = 'S&P 500', yaxis='y2')

hotelfig.update_layout(
      title_x=0.5,
      yaxis=dict(title='AF Hotels Index', side='left', range=[50,210]),
      yaxis2=dict(title='S&P 500', overlaying='y', side='right', range=[2000,6200]),
      xaxis=dict(title='Date'),
      legend=dict(x=0.01, y=0.99)
)


  hotelfig.show()
